In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv("train.csv")

In [42]:
df_train[["PassengerId", "Transported"]].head(5)

,PassengerId,Transported
0,0001_01,False
1,0002_01,True
2,0003_01,False
3,0003_02,False
4,0004_01,True


In [ ]:
df_train[["PassengerId"]]

In [59]:
df_train["travelingInGroup"] = df_train['PassengerId'].str[0:4].duplicated(keep=False).astype('int')

In [21]:
df_train['Cabin_lbl'] = pd.Categorical(df_train['Cabin'].str[0]).codes

In [31]:
df_train["TotalSpendings"] = df_train[['RoomService' , 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck' ]].sum(axis=1)

In [99]:
def X_test(df_train):
    df_train["travelingInGroup"] = df_train['PassengerId'].str[0:4].duplicated(keep=False).astype('int')
    df_train["TotalSpendings"] = df_train[['RoomService' , 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck' ]].sum(axis=1)
    df_train['Cabin_lbl'] = pd.Categorical(df_train['Cabin'].str[0]).codes
    features = ["HomePlanet", "CryoSleep", "Destination"] #"RoomService_bins","FoodCourt_bins","ShoppingMall_bins","VRDeck_bins", "Spa_bins", "VIP"
    X = pd.get_dummies(df_train[features]).join(df_train[['RoomService', 'FoodCourt','ShoppingMall','Spa','VRDeck', "TotalSpendings","travelingInGroup",'Cabin_lbl']])#'Cabin_lbl'
    return X

In [ ]:
df_train['PassengerId'].fillna(value=)

In [259]:
#Id with missing HomePlanet value but traveling together
list_missing_home = df_train[df_train['HomePlanet'].isna() == True].query('travelingInGroup == 1')['PassengerId'].str[0:4]
# get all values as list for each id 

In [278]:
list_missing_home_index = df_train[df_train['HomePlanet'].isna() == True].query('travelingInGroup == 1').index

In [275]:
df_train[df_train['HomePlanet'].isna() == True].index

Int64Index([  59,  113,  186,  225,  234,  274,  286,  291,  347,  365,
            ...
            8353, 8383, 8454, 8468, 8489, 8515, 8613, 8666, 8674, 8684],
           dtype='int64', length=201)

In [196]:
df_train['Psg_group'] = df_train['PassengerId'].str[0:4]

In [199]:
df_train[['PassengerId','HomePlanet','Psg_group']][df_train['PassengerId'].str.contains('|'.join(list_missing_home))]

,PassengerId,HomePlanet,Psg_group
58,0064_01,Mars,0064
59,0064_02,NaN,0064
113,0119_01,NaN,0119
114,0119_02,Europa,0119
404,0444_01,Mars,0444
...,...,...,...
8489,9072_01,NaN,9072
8490,9072_02,Mars,9072
8491,9072_03,Mars,9072
8613,9194_01,NaN,9194


In [277]:
df_train.query("Psg_group == '0210'")['HomePlanet'].sort_values().tolist()

[nan]

In [237]:
counter = 0
df_train['Psg_group'].iloc[counter]
if type(planet) == float:
        df_train['HomePlanet'].iloc[counter] #= df_train.query(f"Psg_group == {counter}")['HomePlanet'].sort_values().tolist()[0]
        id = df_train['Psg_group'].iloc[counter]
        #print(df_train['HomePlanet'].iloc[counter])
        print(id)
        print(df_train.query(f"Psg_group == '{id}'")['HomePlanet'].sort_values().tolist()[0])

'Europa'

In [284]:

for id in list_missing_home_index:
    room = df_train['Psg_group'].iloc[id]
    df_train['HomePlanet'].iloc[id].fillna(df_train.query(f"Psg_group == '{room}'")['HomePlanet'].sort_values().tolist()[0], inplace = True)
    


AttributeError: 'str' object has no attribute 'fillna'

In [ ]:
df_train[df_train['HomePlanet'].isna() == True].query('travelingInGroup == 1')['PassengerId']

In [94]:
features = ["HomePlanet", "CryoSleep", "Destination"] #"RoomService_bins","FoodCourt_bins","ShoppingMall_bins","VRDeck_bins", "Spa_bins", "VIP"
X = pd.get_dummies(df_train[features]).join(df_train[['RoomService', 'FoodCourt','ShoppingMall','Spa','VRDeck', "TotalSpendings","travelingInGroup",'Cabin_lbl']])#'Cabin_lbl'
Y = df_train['Transported']

In [288]:
X.head()

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,TotalSpendings,travelingInGroup,Cabin_lbl
0,0,1,0,1,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,1,0,0,1,0,0,0,1,109.0,9.0,25.0,549.0,44.0,736.0,0,5
2,0,1,0,1,0,0,0,1,43.0,3576.0,0.0,6715.0,49.0,10383.0,1,0
3,0,1,0,1,0,0,0,1,0.0,1283.0,371.0,3329.0,193.0,5176.0,1,0
4,1,0,0,1,0,0,0,1,303.0,70.0,151.0,565.0,2.0,1091.0,0,5


In [245]:
np.save('x_train.npy', X.to_numpy(), allow_pickle=True)
np.save('y_train.npy', Y.to_numpy(), allow_pickle=True)

In [5]:
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [170]:
x_train, x_cv, y_train, y_cv = train_test_split(X,Y,test_size=0.20, random_state=1)

dtrain = xgb.DMatrix(x_train,y_train)
dtest = xgb.DMatrix(x_cv)

param = {'booster': 'gbtree', 'eval_metric': 'auc', 'learning_rate': 0.4, 'max_depth':5, 'objective': 'binary:logistic'}#{"eval_metric":"auc", "objective": "binary:logistic"}
num_rounds = 10
model = xgb.train(param, dtrain, num_rounds)

preds = model.predict(dtest)

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_cv, preds.round())
print(accuracy)

0.8016101207590569


In [75]:
#making predictions

In [100]:
df_test = pd.read_csv("test.csv")

In [101]:
X_pred = X_test(df_test)

In [103]:
dpred = xgb.DMatrix(X_pred)

In [161]:
tst_preds = model.predict(dpred)

In [81]:
#creating CSV

In [162]:
df_subm = pd.DataFrame({"PassengerId":df_test["PassengerId"].tolist(),"Transported": tst_preds.round().tolist()})
df_subm["Transported"] = df_subm["Transported"].astype('bool')
df_subm.to_csv("submission_XGboost_v2.csv",index=False)

In [163]:
df_subm.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [166]:
df_subm_v0 = pd.read_csv('submission_XGboost_v1.csv')

In [167]:
diff_rows = (df_subm.ne(df_subm_v0)).any(axis=1)

In [168]:
diff_rows.value_counts()

False    4077
True      200
dtype: int64

In [113]:
103/4174

0.02467656923814087

In [286]:
# trying out the catboost
from catboost import CatBoostClassifier

In [320]:
cat_model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, loss_function='Logloss', 
                           eval_metric='AUC', use_best_model=True, random_seed=42) #

In [321]:
# fit the model on the training data
x_train, x_cv, y_train, y_cv = train_test_split(X,Y,test_size=0.10, random_state=1)
cat_model.fit(x_train, y_train, eval_set=(x_cv, y_cv), early_stopping_rounds=20) #cat_features=[0,1,2], 

0:	test: 0.8465868	best: 0.8465868 (0)	total: 2.89ms	remaining: 2.88s
1:	test: 0.8665027	best: 0.8665027 (1)	total: 5.87ms	remaining: 2.93s
2:	test: 0.8700673	best: 0.8700673 (2)	total: 8.34ms	remaining: 2.77s
3:	test: 0.8754367	best: 0.8754367 (3)	total: 11.6ms	remaining: 2.88s
4:	test: 0.8749108	best: 0.8754367 (3)	total: 14.2ms	remaining: 2.82s
5:	test: 0.8761210	best: 0.8761210 (5)	total: 16.6ms	remaining: 2.75s
6:	test: 0.8781610	best: 0.8781610 (6)	total: 18.7ms	remaining: 2.65s
7:	test: 0.8766020	best: 0.8781610 (6)	total: 20.9ms	remaining: 2.59s
8:	test: 0.8781187	best: 0.8781610 (6)	total: 23.5ms	remaining: 2.59s
9:	test: 0.8790620	best: 0.8790620 (9)	total: 25.7ms	remaining: 2.54s
10:	test: 0.8799367	best: 0.8799367 (10)	total: 28.1ms	remaining: 2.52s
11:	test: 0.8834828	best: 0.8834828 (11)	total: 30.4ms	remaining: 2.5s
12:	test: 0.8846508	best: 0.8846508 (12)	total: 33.4ms	remaining: 2.53s
13:	test: 0.8865956	best: 0.8865956 (13)	total: 35.5ms	remaining: 2.5s
14:	test: 0.88

In [322]:
cat_pred = cat_model.predict(X_pred)

In [309]:
cat_pred.shape

(4277,)

In [323]:
df_subm = pd.DataFrame({"PassengerId":df_test["PassengerId"].tolist(),"Transported":cat_pred.ravel().tolist()})
#df_subm["Transported"] = df_subm["Transported"].astype('bool')
df_subm.to_csv("submission_catboost_V4.csv",index=False)

In [324]:
df_subm.head(100)

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
95,0214_02,False
96,0215_01,True
97,0218_01,False
98,0226_01,True
